In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
import PIL
import os
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Enter directory from which images are to be segmented
directory= '/content/drive/MyDrive/UGV_AI/tomato/val'
#Traverse each file in the directory
for filename in os.listdir(directory):
  print(filename)
  c= 0
  for f in os.listdir(directory + '/' + filename):
        #Initialize counter to name the images
        c= int(c)
        c= c+ 1
        c= str(c)
        path= directory + '/' + filename + '/' + f
        print(path)
        #Read each image
        image= cv.imread(path)
        gray = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
        #Otsu's thresholding
        ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
        #Noise removal
        kernel = np.ones((2,2),np.uint8)
        opening = cv.morphologyEx(thresh,cv.MORPH_CLOSE,kernel, iterations = 2)
        opening= np.uint8(opening)
        #Erosion
        sure_fg = cv.erode(opening,kernel,iterations=2)
        sure_fg = np.uint8(sure_fg)
        #Apply watershed algorithm
        ret, markers = cv.connectedComponents(sure_fg)
        markers= markers+1
        markers= cv.watershed(image,markers)
        image[markers == 1] = [0, 0, 0]
        cv2_imshow(image)
        #Save the images in the given path
        path= '/content/drive/MyDrive/UGV_AI/tomato/segmented_val' + '/' + filename
        cv.imwrite(os.path.join(path , c+ '.jpg'), image)
